<a href="https://colab.research.google.com/github/lugastyan98/portofolio/blob/main/rekomendation_system_versi_berdasarkan_produk_perusahaan_dsb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# library

In [ ]:
!pip install rake_nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import rake_nltk
from rake_nltk import Rake
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from bs4 import BeautifulSoup #untuk perHTMLan

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Load dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/PINA dataset/semua_versi_4___edwin_2023-05-10T04_25_24.475027Z.csv')

In [ ]:
df_user = pd.read_csv('/content/drive/MyDrive/PINA dataset/user_portfolio.csv')

In [ ]:
df_user.head()

,clientportfolioclientid,clientportfoliostockid,clientportfoliobalance,clientportfoliodate,date_rank_per_user
0,0000003,FIMP,19000000.0,2023-05-05,1
1,01OLK,BOLT,100.0,2023-05-05,1
2,3,CASH,47051800.0,2022-03-28,1
3,ADANGWAH107,TLKM,100.0,2023-05-05,1
4,ADIN99,TRIM,100.0,2023-05-05,1


In [ ]:
df_grouped = pd.DataFrame(df_user.groupby('clientportfolioclientid')['clientportfoliostockid'].agg(list))

In [ ]:
df_grouped = df_grouped.reset_index()

In [ ]:
df_grouped.head()

,clientportfolioclientid,clientportfoliostockid
0,0000003,[FIMP]
1,01OLK,[BOLT]
2,3,[CASH]
3,ADANGWAH107,[TLKM]
4,ADIN99,[TRIM]


In [ ]:
df_grouped.columns  = ['id','portofolio']

In [ ]:
df_grouped.head()

,id,portofolio
0,0000003,[FIMP]
1,01OLK,[BOLT]
2,3,[CASH]
3,ADANGWAH107,[TLKM]
4,ADIN99,[TRIM]


In [ ]:
df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555 entries, 0 to 554
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          555 non-null    object
 1   portofolio  555 non-null    object
dtypes: object(2)
memory usage: 8.8+ KB


In [ ]:
print(df_grouped['id'].nunique())

555


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883 entries, 0 to 882
Data columns (total 54 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      883 non-null    object 
 1   sector                   883 non-null    object 
 2   idx                      883 non-null    object 
 3   thematic                 883 non-null    object 
 4   general_info             865 non-null    object 
 5   business_product         669 non-null    object 
 6   total_asset_growth       853 non-null    float64
 7   price_sales_ratio        845 non-null    float64
 8   sales_growth             834 non-null    float64
 9   quick_ratio              849 non-null    float64
 10  cost_of_goods_sold       853 non-null    float64
 11  cash_flow_per_share      850 non-null    float64
 12  net_income_growth        839 non-null    float64
 13  operating_profit         853 non-null    float64
 14  income_before_tax        8

# Missing Value

Missing value akan diisi dengan rata-rata.
Tidak dihapus karena data _id itu merupakan data semua saham di PINA


In [ ]:
df.isnull().sum() #cek yang NaNT

_id                          0
sector                       0
idx                          0
thematic                     0
general_info                18
business_product           214
total_asset_growth          30
price_sales_ratio           38
sales_growth                49
quick_ratio                 34
cost_of_goods_sold          30
cash_flow_per_share         33
net_income_growth           44
operating_profit            30
income_before_tax           30
total_assets                30
total_equity                30
total_liabilities           30
debt_to_equity_ratio        30
cash_equivalent             30
total_sales                 30
operating_profit_margin     35
gross_profit_margin         35
net_profit_margin           35
net_income                  30
market_cap                  22
dailyReturn                 22
monthlyReturn               22
annualReturn                22
threeYearReturn             22
fiveYearReturn              22
tenYearReturn               94
listedSh

In [ ]:
#mengisi NaN kolum general_info dengan string kosong
df['general_info'].fillna('', inplace=True)

In [ ]:
count = df['_id'].nunique()
print(count)

883


In [ ]:
df.fillna(df.mean(), inplace=True) #mengisi dengan nilai rata-rata

<ipython-input-19-9598921fdd30>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.fillna(df.mean(), inplace=True) #mengisi dengan nilai rata-rata


In [ ]:
df.info()

In [ ]:
df.columns

Index(['_id', 'sector', 'idx', 'thematic', 'general_info', 'business_product',
       'total_asset_growth', 'price_sales_ratio', 'sales_growth',
       'quick_ratio', 'cost_of_goods_sold', 'cash_flow_per_share',
       'net_income_growth', 'operating_profit', 'income_before_tax',
       'total_assets', 'total_equity', 'total_liabilities',
       'debt_to_equity_ratio', 'cash_equivalent', 'total_sales',
       'operating_profit_margin', 'gross_profit_margin', 'net_profit_margin',
       'net_income', 'market_cap', 'dailyReturn', 'monthlyReturn',
       'annualReturn', 'threeYearReturn', 'fiveYearReturn', 'tenYearReturn',
       'listedShare', 'tradableVolume', 'offerVolume', 'tradableValue',
       'openPrice', 'closePrice', 'averagePrice', 'lastTradedPrice',
       'lowTradedPrice', 'highTradedPrice', 'bidPrice', 'offerPrice',
       'book_value_per_share', 'eps', 'pbv', 'roa', 'roe', 'per',
       'description', 'underwriter', 'shareholders', 'industry'],
      dtype='object')

In [ ]:
#yang NaN ini diisi sama ''
df[['business_product','description', 'underwriter', 'shareholders']] = df[['business_product','description', 'underwriter', 'shareholders']].fillna('')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883 entries, 0 to 882
Data columns (total 54 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      883 non-null    object 
 1   sector                   883 non-null    object 
 2   idx                      883 non-null    object 
 3   thematic                 883 non-null    object 
 4   general_info             883 non-null    object 
 5   business_product         883 non-null    object 
 6   total_asset_growth       883 non-null    float64
 7   price_sales_ratio        883 non-null    float64
 8   sales_growth             883 non-null    float64
 9   quick_ratio              883 non-null    float64
 10  cost_of_goods_sold       883 non-null    float64
 11  cash_flow_per_share      883 non-null    float64
 12  net_income_growth        883 non-null    float64
 13  operating_profit         883 non-null    float64
 14  income_before_tax        8

In [ ]:
df.columns

Index(['_id', 'sector', 'idx', 'thematic', 'general_info', 'business_product',
       'total_asset_growth', 'price_sales_ratio', 'sales_growth',
       'quick_ratio', 'cost_of_goods_sold', 'cash_flow_per_share',
       'net_income_growth', 'operating_profit', 'income_before_tax',
       'total_assets', 'total_equity', 'total_liabilities',
       'debt_to_equity_ratio', 'cash_equivalent', 'total_sales',
       'operating_profit_margin', 'gross_profit_margin', 'net_profit_margin',
       'net_income', 'market_cap', 'dailyReturn', 'monthlyReturn',
       'annualReturn', 'threeYearReturn', 'fiveYearReturn', 'tenYearReturn',
       'listedShare', 'tradableVolume', 'offerVolume', 'tradableValue',
       'openPrice', 'closePrice', 'averagePrice', 'lastTradedPrice',
       'lowTradedPrice', 'highTradedPrice', 'bidPrice', 'offerPrice',
       'book_value_per_share', 'eps', 'pbv', 'roa', 'roe', 'per',
       'description', 'underwriter', 'shareholders', 'industry'],
      dtype='object')

# Menghapus Kolum tidak penting

In [ ]:
#aku akan menghapus semua yang float karena ini versi menggunakan bacotan okay
columns_to_drop = ['total_asset_growth', 'price_sales_ratio', 'sales_growth',
       'quick_ratio', 'cost_of_goods_sold', 'cash_flow_per_share',
       'net_income_growth', 'operating_profit', 'income_before_tax',
       'total_assets', 'total_equity', 'total_liabilities',
       'debt_to_equity_ratio', 'cash_equivalent', 'total_sales',
       'operating_profit_margin', 'gross_profit_margin', 'net_profit_margin',
       'net_income', 'market_cap', 'dailyReturn', 'monthlyReturn',
       'annualReturn', 'threeYearReturn', 'fiveYearReturn', 'tenYearReturn',
       'listedShare', 'tradableVolume', 'offerVolume', 'tradableValue',
       'openPrice', 'closePrice', 'averagePrice', 'lastTradedPrice',
       'lowTradedPrice', 'highTradedPrice', 'bidPrice', 'offerPrice',
       'book_value_per_share', 'eps', 'pbv', 'roa', 'roe', 'per']
df = df.drop(columns=columns_to_drop)


In [ ]:
df.head()

,_id,sector,thematic,business_product,industry
0,AALI,"[""Non-Siklikal""]",[],<p>Based&#32;on&#32;the&#32;Company&#39;s&#32;...,Produk Makanan Pertanian
1,ABBA,"[""Siklikal""]",[],,Media
2,ABDA,"[""Keuangan""]",[],<p>The&#32;Company&#32;is&#32;engaged&#32;in&#...,Asuransi
3,ABMM,"[""Perindustrian""]","[""Value Investing"" ""Komoditas Supercycle""]",<p>The&#32;Company&#39;s&#32;scope&#32;of&#32;...,Perusahaan Holding Multi-sektor
4,ACES,"[""Siklikal""]","[""Retailer Indonesia"" ""ESG Leaders""]",<p>The&#32;Company&#39;s&#32;scope&#32;of&#32;...,Ritel Khusus


In [ ]:
#ini dihapusin karena terlihat tidak penting, sudah dicek
columns_to_drop = ['idx','general_info','description','underwriter','shareholders']
df = df.drop(columns=columns_to_drop)


KeyError: ignored

In [ ]:
df.head(1)

,_id,sector,thematic,business_product,industry
0,AALI,"[""Non-Siklikal""]",[],<p>Based&#32;on&#32;the&#32;Company&#39;s&#32;...,Produk Makanan Pertanian


# Pengolahan Lebih lanjut

In [ ]:
#menjadikan _id menjadi index
df.set_index('_id', inplace = True)
df.head()

,sector,thematic,business_product,industry
_id,,,,
AALI,"[""Non-Siklikal""]",[],<p>Based&#32;on&#32;the&#32;Company&#39;s&#32;...,Produk Makanan Pertanian
ABBA,"[""Siklikal""]",[],,Media
ABDA,"[""Keuangan""]",[],<p>The&#32;Company&#32;is&#32;engaged&#32;in&#...,Asuransi
ABMM,"[""Perindustrian""]","[""Value Investing"" ""Komoditas Supercycle""]",<p>The&#32;Company&#39;s&#32;scope&#32;of&#32;...,Perusahaan Holding Multi-sektor
ACES,"[""Siklikal""]","[""Retailer Indonesia"" ""ESG Leaders""]",<p>The&#32;Company&#39;s&#32;scope&#32;of&#32;...,Ritel Khusus


In [ ]:
#bacot ah HTML HTMLan, hapus, musnahkan
def html_to_string(html_code):
    soup = BeautifulSoup(html_code, 'html.parser')
    return soup.get_text()


In [ ]:
df['business_product'] = df['business_product'].apply(html_to_string)

In [ ]:
df.head()

,sector,thematic,business_product,industry
_id,,,,
AALI,"[""Non-Siklikal""]",[],Based on the Company's articles of association...,Produk Makanan Pertanian
ABBA,"[""Siklikal""]",[],,Media
ABDA,"[""Keuangan""]",[],The Company is engaged in general insurance bu...,Asuransi
ABMM,"[""Perindustrian""]","[""Value Investing"" ""Komoditas Supercycle""]",The Company's scope of activities are conducti...,Perusahaan Holding Multi-sektor
ACES,"[""Siklikal""]","[""Retailer Indonesia"" ""ESG Leaders""]",The Company's scope of activities consist of g...,Ritel Khusus


In [ ]:
df['business_product'][0]

"Based on the Company's articles of association, the scope of its activities is to engage in plantation operation, general trading, manufacturing, transportation, consultation and services."

In [ ]:
df.head(1)

,sector,thematic,business_product,industry
_id,,,,
AALI,"[""Non-Siklikal""]",[],Based on the Company's articles of association...,Produk Makanan Pertanian


## Mendapatkan keyword dari business product

In [ ]:
df['key_words_business_product'] = ''  # Menetapkan nilai awal untuk kolom 'key_words'

for index, row in df.iterrows():
    plot = row['business_product']
    r = Rake()
    r.extract_keywords_from_text(plot)
    key_words_dict_scores = r.get_word_degrees()
    df.at[index, 'key_words_business_product'] = list(key_words_dict_scores.keys())



# Sector dihilangkan simbol simbolnya

In [ ]:
df['sector'] = df['sector'].str.replace('[', '').str.replace('"', '').str.replace(']', '')
df.head()

<ipython-input-210-c1d3450769de>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['sector'] = df['sector'].str.replace('[', '').str.replace('"', '').str.replace(']', '')


,sector,thematic,business_product,industry,key_words_business_product
_id,,,,,
AALI,non-siklikal,,Based on the Company's articles of association...,Produk Makanan Pertanian,"[based, company, articles, association, scope,..."
ABBA,siklikal,,,Media,[]
ABDA,keuangan,,The Company is engaged in general insurance bu...,Asuransi,"[company, engaged, general, insurance, busines..."
ABMM,perindustrian,Value Investing Komoditas Supercycle,The Company's scope of activities are conducti...,Perusahaan Holding Multi-sektor,"[company, scope, activities, conducting, busin..."
ACES,siklikal,Retailer Indonesia ESG Leaders,The Company's scope of activities consist of g...,Ritel Khusus,"[company, scope, activities, consist, general,..."


In [ ]:
df['sector'] = df['sector'].str.lower()

In [ ]:
df['sector'].unique()

array(['non-siklikal', 'siklikal', 'keuangan', 'perindustrian',
       'infrastruktur', 'properti', 'barang baku', 'energi',
       'transportasi', 'teknologi', '', 'kesehatan',
       'non-siklikal siklikal'], dtype=object)

In [ ]:
df['sector'] = df['sector'].map(lambda x : x.lower().split(' '))

In [ ]:
df.head()

,sector,business_product,industry,key_words_business_product
_id,,,,
AALI,[non-siklikal],Based on the Company's articles of association...,Produk Makanan Pertanian,"[based, company, articles, association, scope,..."
ABBA,[siklikal],,Media,[]
ABDA,[keuangan],The Company is engaged in general insurance bu...,Asuransi,"[company, engaged, general, insurance, busines..."
ABMM,[perindustrian],The Company's scope of activities are conducti...,Perusahaan Holding Multi-sektor,"[company, scope, activities, conducting, busin..."
ACES,[siklikal],The Company's scope of activities consist of g...,Ritel Khusus,"[company, scope, activities, consist, general,..."


In [ ]:
df['sector'][400]

['barang', 'baku']

# Thematic menghilangkan simbol simbol

In [ ]:
df['thematic'] = df['thematic'].str.replace('[', '').str.replace('"', '').str.replace(']', '')

<ipython-input-208-8b033f66425a>:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['thematic'] = df['thematic'].str.replace('[', '').str.replace('"', '').str.replace(']', '')


In [ ]:
df['thematic'][80]

''

In [ ]:
empty_count = df['thematic'].value_counts()['']
print(empty_count)

746


In [ ]:
non_empty_count = df['thematic'].notna().sum()

print(non_empty_count)

883


In [ ]:
883 - 746

137

In [ ]:
df['thematic'].nunique()

df['thematic'].unique()

array(['', 'Value Investing Komoditas Supercycle',
       'Retailer Indonesia ESG Leaders', 'Infrastruktur +62',
       'High Dividend', 'Komoditas Supercycle',
       'Komoditas Supercycle High Dividend Saham Syariah ', 'ESG Leaders',
       'Kecil Tapi Kuat!',
       'Saham Syariah  Komoditas Supercycle High Dividend',
       'Sektor Properti', 'Sektor Terbesar IHSG: BANKS! Terbesar di IDX',
       'High Dividend Terbesar di IDX', 'ESG Leaders Sektor Properti',
       'Technology for The Future', 'Saham Syariah ',
       'High Dividend ESG Leaders Sektor Terbesar IHSG: BANKS! Terbesar di IDX',
       'Sektor Terbesar IHSG: BANKS!',
       'Sektor Terbesar IHSG: BANKS! Value Investing', 'Indonesia Sehat',
       'Sektor Terbesar IHSG: BANKS! Saham Syariah ',
       'ESG Leaders Saham Syariah ',
       'Saham Syariah  Terbesar di IDX Komoditas Supercycle',
       'Indonesia FMCG', 'High Dividend Saham Syariah  Terbesar di IDX',
       'Terbesar di IDX Technology for The Future',
      

In [ ]:
columns_to_drop = ['thematic']
df = df.drop(columns=columns_to_drop)

In [ ]:
#thmeatic dihapus saja

In [ ]:
df.head()

,sector,business_product,industry,key_words_business_product
_id,,,,
AALI,[non-siklikal],Based on the Company's articles of association...,Produk Makanan Pertanian,"[based, company, articles, association, scope,..."
ABBA,[siklikal],,Media,[]
ABDA,[keuangan],The Company is engaged in general insurance bu...,Asuransi,"[company, engaged, general, insurance, busines..."
ABMM,[perindustrian],The Company's scope of activities are conducti...,Perusahaan Holding Multi-sektor,"[company, scope, activities, conducting, busin..."
ACES,[siklikal],The Company's scope of activities consist of g...,Ritel Khusus,"[company, scope, activities, consist, general,..."


# Industri mau diapain nih

In [ ]:
df['industry'].nunique()

59

In [ ]:
df['industry'].unique()

array(['Produk Makanan Pertanian', 'Media', 'Asuransi',
       'Perusahaan Holding Multi-sektor', 'Ritel Khusus',
       'Konstruksi Bangunan', 'Pengelola & Pengembang Real Estat',
       'Minuman', 'Pembiayaan Konsumen', 'Barang Kimia', 'Batu Bara',
       'Bank', 'Makanan Olahan', 'Pariwisata & Rekreasi', 'Minyak & Gas',
       'Logistik & Pengantaran', 'Wadah & Kemasan', 'Logam & Mineral',
       'Produk & Perlengkapan Bangunan', 'Machinery', 'Jasa Investasi',
       'Perdagangan Ritel Barang Primer',
       'Pendukung Minyak, Gas & Batu Bara',
       'Perusahaan Holding & Investasi', 'Pakaian & Barang Mewah',
       'Utilitas Listrik', 'Jasa Komersial',
       'Pengangkutan Darat Penumpang', 'Jasa & Konsultan TI',
       'Komponen Otomotif', 'Aplikasi & Jasa Internet',
       'Perangkat Komputer', 'Material Konstruksi',
       'Jasa Telekomunikasi Nirkabel',
       'Peralatan Olah Raga & Barang Hobi', 'Pendidikan & Jasa Penunjang',
       'Penyedia Jasa Kesehatan', 'Operator Infras

In [ ]:
empty_count = df['industry'].value_counts()['Pendukung Minyak, Gas & Batu Bara']
print(empty_count)

17


In [ ]:
df['industry'] = df['industry'].map(lambda x:x.split(' '))
for index, row in df.iterrows():
  row['industry'] = ''.join(row['industry']).lower() #disatukan supaya diperlakukan sebagai satu objek

In [ ]:
df['industry'] = df['industry'].map(lambda x : x.split(' '))

In [ ]:
df['industry'].unique()

array(['produkmakananpertanian', 'media', 'asuransi',
       'perusahaanholdingmulti-sektor', 'ritelkhusus',
       'konstruksibangunan', 'pengelola&pengembangrealestat', 'minuman',
       'pembiayaankonsumen', 'barangkimia', 'batubara', 'bank',
       'makananolahan', 'pariwisata&rekreasi', 'minyak&gas',
       'logistik&pengantaran', 'wadah&kemasan', 'logam&mineral',
       'produk&perlengkapanbangunan', 'machinery', 'jasainvestasi',
       'perdaganganritelbarangprimer', 'pendukungminyak,gas&batubara',
       'perusahaanholding&investasi', 'pakaian&barangmewah',
       'utilitaslistrik', 'jasakomersial', 'pengangkutandaratpenumpang',
       'jasa&konsultanti', 'komponenotomotif', 'aplikasi&jasainternet',
       'perangkatkomputer', 'materialkonstruksi',
       'jasatelekomunikasinirkabel', 'peralatanolahraga&baranghobi',
       'pendidikan&jasapenunjang', 'penyediajasakesehatan',
       'operatorinfrastrukturtransportasi', 'barangrumahtangga',
       'kelistrikan', 'maskapaipenerban

In [ ]:
df.head(1)

,sector,industry,key_words_business_product,bag_of_words
_id,,,,
AALI,[non-siklikal],[produkmakananpertanian],"[based, company, articles, association, scope,...",non-siklikal p r o d u k m a k a n a n p e r t...


## Menghapus business_product

In [ ]:
columns_to_drop = ['business_product']
df = df.drop(columns=columns_to_drop)

In [ ]:
df.head()

,sector,industry,key_words_business_product
_id,,,
AALI,[non-siklikal],produkmakananpertanian,"[based, company, articles, association, scope,..."
ABBA,[siklikal],media,[]
ABDA,[keuangan],asuransi,"[company, engaged, general, insurance, busines..."
ABMM,[perindustrian],perusahaanholdingmulti-sektor,"[company, scope, activities, conducting, busin..."
ACES,[siklikal],ritelkhusus,"[company, scope, activities, consist, general,..."


# Bag_of_words (kumpulkan semua kata)

In [ ]:
df['bag_of_words'] = ''
columns = df.columns
for index, row in df.iterrows():
  words= ''
  for col in columns:
      words = words + ' '.join(row[col]) + ' '

  row['bag_of_words'] = words


In [ ]:
df.head(3)

,sector,industry,key_words_business_product,bag_of_words
_id,,,,
AALI,[non-siklikal],[produkmakananpertanian],"[based, company, articles, association, scope,...",non-siklikal produkmakananpertanian based comp...
ABBA,[siklikal],[media],[],siklikal media
ABDA,[keuangan],[asuransi],"[company, engaged, general, insurance, busines...",keuangan asuransi company engaged general insu...


In [ ]:
df['bag_of_words'][0]

'non-siklikal produkmakananpertanian based company articles association scope activities engage plantation operation general trading manufacturing transportation consultation services'

## Menghapus double spase

In [ ]:
df['bag_of_words'] = df['bag_of_words'].str.replace(r'\s\s', ' ')

<ipython-input-278-df9588f9567a>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['bag_of_words'] = df['bag_of_words'].str.replace(r'\s\s', ' ')


In [ ]:
#mengecek ada berapa baris yang mengandung double space
df['bag_of_words'].str.contains(r'\s\s').sum()

0

## Menghapus spase yang ada di belakang kalimat

In [ ]:
#menghapus karakter spasi diakhir kaliamt
df['bag_of_words'] = df['bag_of_words'].str.rstrip()

## Menghapus karakter khusus

In [ ]:
#menghilangkan karakter khusus
df['bag_of_words'] = df['bag_of_words'].str.replace('[^\w\s]', '', regex=True)

In [ ]:
df.head(1)

,sector,bag_of_words
_id,,
AALI,[non-siklikal],nonsiklikal produkmakananpertanian based compa...


## Hapus sektor dan **kew_word_business_product**

In [ ]:
df = df.drop(columns=['industry','key_words_business_product'])

In [ ]:
df = df.drop(columns=['sector'])

0

<ipython-input-150-df9588f9567a>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['bag_of_words'] = df['bag_of_words'].str.replace(r'\s\s', ' ')


In [ ]:
df.head()

,bag_of_words
_id,
AALI,nonsiklikal produkmakananpertanian based compa...
ABBA,siklikal media
ABDA,keuangan asuransi company engaged general insu...
ABMM,perindustrian perusahaanholdingmultisektor com...
ACES,siklikal ritelkhusus company scope activities ...


# CountVectorize

In [ ]:
count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])

In [ ]:
count_matrix

<883x2742 sparse matrix of type '<class 'numpy.int64'>'
	with 13517 stored elements in Compressed Sparse Row format>

In [ ]:
c = count_matrix.todense()
c

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
print(count_matrix[0,:])

  (0, 1651)	1
  (0, 1975)	1
  (0, 275)	1
  (0, 569)	1
  (0, 217)	1
  (0, 236)	1
  (0, 2202)	1
  (0, 93)	1
  (0, 863)	1
  (0, 1868)	1
  (0, 1708)	1
  (0, 1094)	1
  (0, 2551)	1
  (0, 1500)	1
  (0, 2569)	1
  (0, 625)	1
  (0, 2238)	1


# Menghitung nilai cosine similarity

In [ ]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.        , 0.13453456, ..., 0.09335201, 0.06726728,
        0.2300895 ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.09805807,
        0.        ],
       [0.13453456, 0.        , 1.        , ..., 0.1067521 , 0.11538462,
        0.0877058 ],
       ...,
       [0.09335201, 0.        , 0.1067521 , ..., 1.        , 0.1067521 ,
        0.12171612],
       [0.06726728, 0.09805807, 0.11538462, ..., 0.1067521 , 1.        ,
        0.0877058 ],
       [0.2300895 , 0.        , 0.0877058 , ..., 0.12171612, 0.0877058 ,
        1.        ]])

# Membuat rekomendasi

In [ ]:
indices = pd.Series(df.index)
indices

0      AALI
1      ABBA
2      ABDA
3      ABMM
4      ACES
       ... 
878    ZATA
879    ZBRA
880    ZINC
881    ZONE
882    ZYRX
Name: _id, Length: 883, dtype: object

In [ ]:
def recomendations(id, cosine_sim = cosine_sim):
  recomended_stock= []
  idx = indices[indices == id].index[0]

  score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

  top_10_indexes = list(score_series.iloc[1:11].index)

  print(top_10_indexes)

  for i in top_10_indexes:
    recomended_stock.append(list(df.index)[i])

  return recomended_stock

In [ ]:
recomendations('MAGP')

[177, 656, 724, 849, 0, 604, 255, 496, 782, 261]


['BWPT',
 'PSGO',
 'SIMP',
 'VICO',
 'AALI',
 'PALM',
 'DSFI',
 'MAIN',
 'TAXI',
 'DWGL']

In [ ]:
df['bag_of_words'].loc['MAGP']

'nonsiklikal produkmakananpertanian scope activities company mainly engage oil palm plantation'

In [ ]:
df['bag_of_words'].loc['BWPT']

'nonsiklikal produkmakananpertanian scope company activities mainly engage manufacturing agricultural business subsidiaries incorporated conduct operations indonesia currently engages palm plantation products consist refined crude oil kernel'

In [ ]:
df['bag_of_words'].loc['PSGO']

'nonsiklikal produkmakananpertanian company  main scope activities engaged holding management consultancy trading currently plantation palm oil processing industriesthrough subsidiary'

In [ ]:
df['bag_of_words'].loc['SIMP']

'nonsiklikal produkmakananpertanian company scope activities agribusiness plantation manufactures edible oils fats'

In [ ]:
df['bag_of_words'].loc['AALI']

'nonsiklikal produkmakananpertanian based company articles association scope activities engage plantation operation general trading manufacturing transportation consultation services'

In [ ]:
df['bag_of_words'].loc['PALM']

'nonsiklikal produkmakananpertanian scope company activities engage agriculture trading manufacturing transportation services relation agroindustry'

In [ ]:
df['bag_of_words'].loc['DSFI']

'nonsiklikal produkmakananpertanian scope company activities engage fishery including catching processing selling trading products'

In [ ]:
df['bag_of_words'].loc['MAIN']

'nonsiklikal produkmakananpertanian company scope activities engage poultry feeds industry day old chicken doc farm'

In [ ]:
df['bag_of_words'].loc['TAXI']

'transportasi pengangkutandaratpenumpang scope company activities mainly engage land transportation services'

In [ ]:
df['bag_of_words'].loc['DWGL']

'energi batubara scope company activities engage mainly mining trading coal'